In [ ]:
#calling dependencies

import sys
import nltk
import pandas
import sklearn
import numpy

print('Python:{}',format(sys.version))
print('NLTK:{}',format(nltk.__version__))
print('Pandas:{}',format(pandas.__version__))
print('Scikit-learn:{}',format(sklearn.__version__))
print('Numpy:{}',format(numpy.__version__))

      


loading the Dataset

In [ ]:
import pandas as pd
import numpy as np

#load  dataset in workable format

df = pd.read_table('SMSSpamCollection',header = None, encoding = 'utf-8')




In [ ]:
#basic info about the dataset

print(df.info())
print(df.head())
print(
)
classes = df[0]
print(classes.value_counts())

2. preprocess the data

convert class labels to binary values 0=ham, 1= spam

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
Y = encoder.fit_transform(classes)
print(classes[:10])
print(Y[:10])

store sms message data

In [ ]:
sms_messages = df[1]
print(sms_messages[:15])

use regexp to replace email addresses, url's, phone numbers, other numbers, symbols

replace email addresses with 'emailadd'


In [ ]:
#ue regexp to replace urls, email addresses and currencies
processed = sms_messages.str.replace(r'^,+@[^\,],*\,[a-z]{2,}$', 'emilad')  

processed = processed.str.replace(r'^http\://[a-zA-Z0-9\-\,]+\,[a-zA-Z]{2,3}(/\S*)?$', 'wbadd')

processed = processed.str.replace(r'£|\$', 'cshsgn')

processed = processed.str.replace(r'^\(?[\d]{3}\)?[\s-]?[\d]{3}[\s-]?[\d]{4}$','pnumbr')
                                  
processed = processed.str.replace(r'\d+(\.\d+)?', 'nber')


In [ ]:
#remove punctuation

processed = processed.str.replace(r'[^\w\d\s]','')

#remove whitespace between terms
#processed = processed.str.replace(r'\s+','')

#remove leading and trailing whitespace

#processed = processed.str.replace(r'^\s+|\s+?$', '')

In [ ]:
#change to lower case
processed = processed.str.lower()
print(processed)


In [ ]:
#remove stopwords from txt
#from nltk.corpus import stopwords

#stop_words = set(stopwords.words('english'))
#print(stop_words)


#temp_processed=processed.split(' ')



#temp_processed=[token for token in processed if token not in stop_words]
#print (temp_processed)


#processed = processed.apply(lambda x: ''.join(term for term in x.split() if term not in stop_words))

In [ ]:
#remove word stems using porter stemming


#from nltk.stem import PorterStemmer
#from nltk import pos_tag

#ps = nltk.PorterStemmer()


#processed = processed.apply(lambda x: ''.join(ps.stem(term) for term in x.split()))

In [ ]:
print (processed)

In [ ]:
#feature engineering

from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.probability import FreqDist

#creating bag of words

all_words = []

#for message in processed:
   # words = nltk.tokenize.word_tokenize(message)

for message in processed:
    sentences = sent_tokenize(message)
    for words in sentences:
        w= word_tokenize(words)
        for c in w:
            all_words.append(c)
    
all_words = FreqDist(all_words)

    # for word in word_tokenize(sentence):
           # fdist[word] += 1
            
#all_words=nltk.FreqDist(all_words)

#words = nltk.tokenize.word_tokenize(processed)
#all_words= FreqDist(words)




In [ ]:
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))
#print(stop_words)


#temp_processed=processed.split(' ')



temp_processed=[token for token in all_words if token not in stop_words]

#print (temp_processed)
temp_processed = FreqDist(temp_processed)
print('number of words: {}',format(len(temp_processed)))
print('Most common words: {}',format(temp_processed.most_common(3000)))

In [ ]:

print('number of words: {}',format(len(temp_processed)))
print('Most common words: {}',format(temp_processed.most_common(3000)))
      

In [ ]:
#use the most common features to train
word_features = list(all_words.keys())[:1500]

In [ ]:
#define a find features function
def find_features(message):
    words=word_tokenize(message)
    features = {}
    for word in word_features:
        features[word] = (word in words)
        
    return features
#lets see results

features = find_features(processed[0])
for key,value in features.items():
    if value == True:
        print (key)

In [ ]:
#find features for all messages
messages = list(zip(processed, Y))


#define seed for reproducibility
seed=1
np.random.seed = seed
np.random.shuffle(messages)

featuresets = [(find_features(text),label) for (text, label) in messages]

In [ ]:
#splitting the featuresets into training and testing
from sklearn import model_selection

#split the data into training and testing
training, testing = model_selection.train_test_split(featuresets, test_size =0.20, random_state=seed)


In [ ]:
print(len(training))
print(len(testing))

Scikit learn Classifiers with NLTK

In [ ]:
#we will import some algorithms from sklearn to use as well as some performance metrics
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.svm import SVC

model= SklearnClassifier(SVC(kernel = 'linear'))


#train the model
model.train(training)

#test on testing dataset
accuracy = nltk.classify.accuracy(model, testing)*100

print("SVC accuracy: {}". format(accuracy))

In [ ]:

from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

#define models to train
names = ["K Nearest Neighbors", "Decision Tree", "Random Forest", "Logistic Regression", 
         "SGD Classifier", "Naive Bayes", "SVM Linear"]

import warnings
warnings.filterwarnings("ignore")

classifiers= [
    DecisionTreeClassifier(),
    KNeighborsClassifier(),
    LogisticRegression(),
    RandomForestClassifier(),
    MultinomialNB(),
    SGDClassifier(max_iter = 100),
    SVC(kernel = 'linear')
]

models = list(zip(names, classifiers))

for name, model in models:
    nltk_model =SklearnClassifier(model)
    nltk_model.train(training)
    accuracy = nltk.classify.accuracy(nltk_model, testing)*100
    print("{} Accuracy: {}".format(name, accuracy))

In [ ]:
#ensemble voting classifier
from sklearn.ensemble import VotingClassifier

names = ["K Nearest Neighbors", "Decision Tree", "Random Forest", "Logistic Regression", 
         "SGD Classifier", "Naive Bayes", "SVM Linear"]

classifiers= [
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    LogisticRegression(),
    SGDClassifier(max_iter = 100),
    MultinomialNB(),
    SVC(kernel = 'linear')
]


models = list(zip(names, classifiers))

nltk_ensemble = SklearnClassifier(VotingClassifier(estimators = models,voting ='hard', n_jobs = -1))
nltk_ensemble.train(training)
accuracy = nltk.classify. accuracy(nltk_model, testing)*100
print("Voting Classifier: Accuracy : {}".format(accuracy))



In [ ]:
#make class label for testing set
testing_features, labels = list(zip(*testing))

prediction = nltk_ensemble.classify_many(testing_features)


In [ ]:
#print a confusion matrix and classification report
print(classification_report(labels, prediction))

pd.DataFrame(
    confusion_matrix(labels, prediction),
    index = [['actual', 'actual'], ['ham', 'spam']],
    columns = [['predicted', 'predicted'], ['ham', 'spam']])